In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Objective: To understand how turnover affects profit in Las Vegas from 2015 - 2022

In [61]:
data = pd.read_csv("data.csv")

In [64]:
data.head()

,Unnamed: 0,Type,Date,Num,Name,Memo,Clr,Split,Amount,Balance
0,CHASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Check,12/17/14,NaN,FRANCHISE TAX BOARD,2014 chase chk #444,X,LLC Tax,-800.0,-800.0
2,NaN,Deposit,12/30/14,NaN,"GoldenWest Management, Inc",1828 BRIGHT LEAF 2014,X,Rental Income,12540.0,11740.0
3,NaN,Check,12/30/14,NaN,"GoldenWest Management, Inc",BRIGHT LEAF 2014,X,Management Fees,-900.0,10840.0
4,NaN,Check,12/30/14,NaN,State Farm Fire and Casualty Company,BRIGHT LEAF 2014,X,Insurance Expense,-532.0,10308.0


In [65]:
data["Date"] = pd.to_datetime(data["Date"])

In [66]:
data["Year"] = (data["Date"].apply(lambda x: x.year))

In [67]:
df = data.loc[data["Year"].isin(np.arange(2015, 2022+1))]
df = df.loc[df["Type"].isin(["Deposit", "Check"])]

In [68]:
df["Memo"].str.lower().str.contains("leaf").isna().sum()

2

In [ ]:
contains leaf or bright in case one is misspelled

In [53]:
df.drop(["Unnamed: 0", "Date", "Num", "Memo", "Clr", "Balance"], axis=1)

,Type,Name,Split,Amount,Year
19,Deposit,"GoldenWest Management, Inc",Rental Income,1050.0,2015.0
20,Check,"GoldenWest Management, Inc",Management Fees,-75.0,2015.0
21,Deposit,"GoldenWest Management, Inc",Rental Income,1550.0,2015.0
22,Check,"GoldenWest Management, Inc",Management Fees,-100.0,2015.0
23,Check,State Farm Fire and Casualty Company,Insurance Expense,-536.0,2015.0
...,...,...,...,...,...
744,Check,"GoldenWest Management, Inc",Management Fees,-175.0,2022.0
745,Check,"GoldenWest Management, Inc",Management Fees,-150.0,2022.0
746,Deposit,"GoldenWest Management, Inc",Management Fees,175.0,2022.0
748,Check,ELDORADO THIRD CA,Community Association Fee,-69.0,2022.0


In [ ]:
Split = ["Rental Income", "Management Fees", "Community Association Fee", "Insurance Expense", "Repairs and Maintenace", "LLC Tax", "Utilities"]

In [ ]:
If looking at community association fee for Fort Pike look at "MOUNTAINS EDGE MASTER ASSOCIATION"

Clark county water reclamation is fort pike